In [2]:
import json
import pandas as pd

# Process the JSON to extract what we need

In [15]:
d = pd.read_json('data/squad2.json')['data']

In [35]:
all_paragraphs = [paragraph['context'] for ent in d for paragraph in ent['paragraphs']]

In [41]:
# Ok. list comprehension is still possible but it gets a little obnoxious.
all_qa = []
for ent in d:
    for paragraph in ent['paragraphs']:
        for qa in paragraph['qas']:
            if len(qa['answers']) > 0 and qa['is_impossible'] == False:
                all_qa.append((qa['question'], qa['answers'][0]['text']))

In [43]:
len(all_qa)

86821

In [44]:
len(all_paragraphs)

19035

In [47]:
all_paragraphs[0:5]

['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring ro

In [49]:
all_qa[0:5]

[('When did Beyonce start becoming popular?', 'in the late 1990s'),
 ('What areas did Beyonce compete in when she was growing up?',
  'singing and dancing'),
 ("When did Beyonce leave Destiny's Child and become a solo singer?", '2003'),
 ('In what city and state did Beyonce  grow up? ', 'Houston, Texas'),
 ('In which decade did Beyonce become famous?', 'late 1990s')]

In [53]:
import pickle

In [58]:
f = open('data/paragraphs_and_qa.pickle','wb')
pickle.dump(all_paragraphs, f)
pickle.dump(all_qa, f)
f.close()